In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm.notebook import tqdm
import pandas as pd
import time
from tqdm.notebook import tqdm
import pickle

In [6]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Chrome/87.0 (Harrando@eurecom.fr) SPARQLWrapper")

In [7]:
classes = {
    "Organization": "Q43229", #includes companies
    "Name": "Q82799",
    "Artist": "Q483501",
    "Geolocation": "Q2221906",
    "City": "Q515",
    "Capital": "Q5119",
    "Town": "Q3957",
    "Demonym": "Q217438",
    "Product": "Q2424752",
    "Brand": "Q431289",
    "Georegion": "Q82794",
    "Country": "Q6256",
}

In [8]:
len(classes)

12

In [9]:
%%time
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
results = {}
limit = 1000000

for label, wikidata_code in tqdm(classes.items()):
    print(label, wikidata_code)
    results[label] = []
    offset = 0
    while True:
        try:
             #  ?entity ?entityLabel ?entityAltLabel ?entityDescription
            query = """
                    SELECT DISTINCT ?entity ?entityLabel ?entityAltLabel ?entityDescription
                    WHERE
                    {{
                      {{
                        SELECT DISTINCT ?entity WHERE {{
                          ?entity wdt:P31|wdt:P31/wdt:P279|wdt:P31/wdt:P279/wdt:P279 wd:{};
                                  wdt:P31 ?class
                        }}
                        OFFSET {}
                        LIMIT {}
                      }}
                      SERVICE wikibase:label {{ bd:serviceParam wikibase:language 'en' }}
                    }}
                    """.format(wikidata_code, offset*limit, limit)
            if offset == 0:
                print(query)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            offset += 1
            if len(sparql.query().convert()['results']['bindings']) > 0:
                result = sparql.query().convert()['results']['bindings']
                results[label].append(result)
                print(f'{len(results[label][-1])} of results returned at offset {offset}')
            else:
                print(f'No more results returned (offset {offset})')
                break
        except Exception as e:
            print(f'Exception at label {label} (offset {offset}) : {e}')

Organization Q43229

                    SELECT DISTINCT ?entity ?entityLabel ?entityAltLabel ?entityDescription
                    WHERE
                    {
                      {
                        SELECT DISTINCT ?entity WHERE {
                          ?entity wdt:P31|wdt:P31/wdt:P279|wdt:P31/wdt:P279/wdt:P279 wd:Q43229;
                                  wdt:P31 ?class
                        }
                        OFFSET 0
                        LIMIT 1000000
                      }
                      SERVICE wikibase:label { bd:serviceParam wikibase:language 'en' }
                    }
                    



KeyboardInterrupt: 

In [ ]:
for label in classes:
    print(label.upper())
    print(len(results[label]), 'results')
    print(len(results_2[label]), 'results')
    print(0 if label not in results_3 else len(results_3[label]), 'results')
    print('')

In [25]:
for label in classes:
    print(label.upper())
    print('About', len(results[label]) * limit, 'results')
    print('')

ORGANIZATION
About 940000 results

NAME
About 40000 results

ARTIST
About 20000 results

GEOLOCATION
About 3100000 results

CITY
About 40000 results

CAPITAL
About 20000 results

TOWN
About 60000 results

DEMONYM
About 20000 results

PRODUCT
About 80000 results

BRAND
About 20000 results

GEOREGION
About 1700000 results

COUNTRY
About 20000 results



In [26]:
# v1.0 working with {0,2}
# v1.1 adding classes QID
pickle.dump(results, open('gazetteers_v1.1_results.pickle', 'wb'))

In [ ]:
'''
SELECT distinct  ?entity (GROUP_CONCAT(?class ; SEPARATOR = ';') as ?classes)
WHERE 
{ 
     ?entity wdt:P31/wdt:P279* wd:Q82799;
             wdt:P31 ?class.
}
GROUP BY ?entity
LIMIT 100
'''

In [15]:
%%time
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
# results_classes = {}
limit = 20000

for label, wikidata_code in tqdm(classes.items()):
    if label not in ['Geolocation', 'Georegion']:
        continue
    print(label, wikidata_code)
    results_classes[label] = []
    offset = 0
    while True:
        try:
             #  ?entity ?entityLabel ?entityAltLabel ?entityDescription
            query = """
                    SELECT distinct  ?entity (GROUP_CONCAT(?class ; SEPARATOR = ';') as ?classes)
                    WHERE 
                    {{
                         ?entity wdt:P31|wdt:P31/wdt:P279 wd:{};
                                 wdt:P31 ?class.
                    }}
                    GROUP BY ?entity
                    OFFSET {}
                    LIMIT {}
                    """.format(wikidata_code, offset*limit, limit)
            if offset == 0:
                print(query)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            offset += 1
            if len(sparql.query().convert()['results']['bindings']) > 0:
                result = sparql.query().convert()['results']['bindings']
                results_classes[label].append(result)
                print(f'{len(results_classes[label][-1])} of results returned at offset {offset}')
            else:
                print(f'No more results returned (offset {offset})')
                break
        except Exception as e:
            if str(e).startswith('EndPointInternalError'):
                e = 'Wikidata TimeoutException'
                offset -= 1
                time.sleep(30)
            print(f'Exception at label {label} (offset {offset}) : {e}')

Geolocation Q2221906

                    SELECT distinct  ?entity (GROUP_CONCAT(?class ; SEPARATOR = ';') as ?classes)
                    WHERE 
                    {
                         ?entity wdt:P31|wdt:P31/wdt:P279 wd:Q2221906;
                                 wdt:P31 ?class.
                    }
                    GROUP BY ?entity
                    OFFSET 0
                    LIMIT 20000
                    
20000 of results returned at offset 1
20000 of results returned at offset 2
20000 of results returned at offset 3
20000 of results returned at offset 4
20000 of results returned at offset 5
20000 of results returned at offset 6
20000 of results returned at offset 7
20000 of results returned at offset 8
20000 of results returned at offset 9
20000 of results returned at offset 10
20000 of results returned at offset 11
20000 of results returned at offset 12
Exception at label Geolocation (offset 12) : Wikidata TimeoutException
20000 of results returned at offset 13
2

20000 of results returned at offset 22
20000 of results returned at offset 23
20000 of results returned at offset 24
20000 of results returned at offset 25
20000 of results returned at offset 26
20000 of results returned at offset 27
20000 of results returned at offset 28
20000 of results returned at offset 29
20000 of results returned at offset 30
20000 of results returned at offset 31
20000 of results returned at offset 32
20000 of results returned at offset 33
20000 of results returned at offset 34
20000 of results returned at offset 35
19625 of results returned at offset 36
No more results returned (offset 37)

CPU times: user 43 s, sys: 7.18 s, total: 50.2 s
Wall time: 3h 44min 34s


In [ ]:
73